In [ ]:
import collections


def solution(arrows):
    answer = 0
    move = [(-1, 0), (-1, 1), (0, 1), (1, 1),
            (1, 0), (1, -1), (0, -1), (-1, -1)]
    now = (0, 0)

    # visited : 노드 방문 체크
    # visited_dir : 노드 방문 경로 체크 ((A, B)는 A -> B 경로를 의미)
    visited = collections.defaultdict(int)
    visited_dir = collections.defaultdict(int)

    # arrows 따라 노드 좌표를 큐에 추가
    queue = collections.deque([now])
    for i in arrows:
        # 모래 시계 형태 예외를 처리하기 위해 해당 방향으로 2칸씩 추가한다.
        for _ in range(2):
            next = (now[0] + move[i][0], now[1] + move[i][1])
            queue.append(next)

            now = next

    now = queue.popleft()
    visited[now] = 1

    while queue:
        next = queue.popleft()

        # 이미 방문한 노드(visited[x]==1)인 경우
        if visited[next] == 1:
            # 해당 경로로 처음 들어온 경우인 경우 answer++
            # 처음 들어온 경우에 방이 처음 생성되므로!
            if visited_dir[(now, next)] == 0:
                answer += 1
        # 처음 방문한 노드인 경우 방문 체크를 한다.
        else:
            visited[next] = 1

        # 해당 노드로 들어온 경로를 방문 체크해준다.
        visited_dir[(now, next)] = 1
        visited_dir[(next, now)] = 1
        now = next

    return answer

In [ ]:
우선, 모래 시계 형태 예외를 처리하기 위해서 한 번 이동할 때 두 칸씩 이동하도록 하여,
기존에 노드가 없는 지점에도 노드가 생기도록 한다.

 

방의 생성 여부는 사이클이 생기는 것을 확인해야 함이 맞지만, 
유니온-파인드 방법을 고수할 필요는 없었다. 방이 생성되는 경우는 다음과 같다.

- 방문한 노드가 이미 방문한 적이 있고,

- 해당 노드로 들어온 경로는 처음 이동한 경우

 

두 번째 조건은 한 경로를 두고 와리가리했을 경우 방이 계속해서 생김을 방지하기 위해 체크한다.

In [ ]:
from collections import defaultdict
def solution(arrows):
    answer = 0
    visit = defaultdict(list)
    x,y = 0,0
    dx,dy = [0,1,1,1,0,-1,-1,-1],[1,1,0,-1,-1,-1,0,1]

    for arrow in arrows:
        for _ in range(2):              # 대각선 판별을 위해 2씩
            nx = x + dx[arrow]
            ny = y + dy[arrow]  
            if (nx,ny) in visit and (x,y) not in visit[(nx,ny)]:    # 방문했던 점이지만 경로가 겹치지 않는 경우, 방+1
                answer += 1
                visit[(x,y)].append((nx,ny))
                visit[(nx,ny)].append((x,y))
            elif (nx,ny) not in visit:                  # 방문하지 않았던 경우
                visit[(x,y)].append((nx,ny))            # 경로가 겹치는 경우는 따로 해줄 필요 없음
                visit[(nx,ny)].append((x,y))
            x,y = nx, ny        # 이동
    return answer

In [ ]:
1) 선을 긋는 것을 어떻게 표현할까?
- 선을 긋는다고 생각하지 말고 다른 방법을 떠올려보자.
2) 방을 찾으려면 어떤 방법을 써야할까?
- 기존의 점과 만날 때 방이 하나씩 생긴다?
3) 대각선이 교차하는 경우는 어떻게 할까?
- " 이동거리를 2로 하면 중간에 노드 하나가 생기게 된다...! 즉 2)의 아이디어를 이용할 수 있다!
4) 왔다갔다 하는 것도 처리

In [ ]:
def solution(arrows):   
    def get_to(curr, move):
        x, y = curr
        x = curr[0] + (move in [1,2,3]) + -1*(move in [5,6,7])
        y = curr[1] + (move in [0,1,7]) + -1*(move in [3,4,5])
        return (x, y)
    def get_crossed(line):
        x1,y1 = line[0]
        x2,y2 = line[1]       
        return tuple(sorted([(x1+1,y1), (x2-1,y2)]))

    rooms = 0
    curr = (0,0)
    lines = set()
    passed = set()
    passed.add(curr)
    for move in arrows:
        to = get_to(curr, move)
        line = tuple(sorted([curr, to]))
        # we've been there last time, so...
        if line in lines:
            curr = to
            continue
        # when "to" has reached the already passed point 
        elif to in passed:
            lines.add(line)
            rooms += 1
        else:
            lines.add(line)
            passed.add(to)
        # any line crossed?
        if line[0][0] != line[1][0] and line[0][1] != line[1][1]:
            if get_crossed(line) in lines:
                rooms += 1
        curr = to
    return rooms
